In [2]:
# Importing libraries
import pandas as pd
from pathlib import Path
import csv
from config import api_key
import requests
from pprint import pprint
import random

import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import plotly.io as pio
pio.renderers.default = "chrome"

In [5]:
csv_path = Path("../Resources/global air pollution dataset.csv")
global_df = pd.read_csv(csv_path)
global_df.head()

,Country,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category
0,Russian Federation,Praskoveya,51,Moderate,1,Good,36,Good,0,Good,51,Moderate
1,Brazil,Presidente Dutra,41,Good,1,Good,5,Good,1,Good,41,Good
2,Italy,Priolo Gargallo,66,Moderate,1,Good,39,Good,2,Good,66,Moderate
3,Poland,Przasnysz,34,Good,1,Good,34,Good,0,Good,20,Good
4,France,Punaauia,22,Good,0,Good,22,Good,0,Good,6,Good


In [6]:
global_df.columns

Index(['Country', 'City', 'AQI Value', 'AQI Category', 'CO AQI Value',
       'CO AQI Category', 'Ozone AQI Value', 'Ozone AQI Category',
       'NO2 AQI Value', 'NO2 AQI Category', 'PM2.5 AQI Value',
       'PM2.5 AQI Category'],
      dtype='object')

In [7]:
NorthAmerica_df = global_df.loc[global_df['Country'].isin(['United States of America', 'Canada', 'Mexico'])]
NorthAmerica_df.head()
NorthAmerica_df.count()

Country               3601
City                  3601
AQI Value             3601
AQI Category          3601
CO AQI Value          3601
CO AQI Category       3601
Ozone AQI Value       3601
Ozone AQI Category    3601
NO2 AQI Value         3601
NO2 AQI Category      3601
PM2.5 AQI Value       3601
PM2.5 AQI Category    3601
dtype: int64

In [8]:
unique_cities = NorthAmerica_df[['Country', 'City']].drop_duplicates().reset_index(drop=True)
cities_list = unique_cities['City']
cities_list.to_csv('../Resources/cities_list.csv')
unique_cities.to_csv('../Resources/citiesandcountry_list.csv')

# Making a list of cities. This is the entire list of 3601 countries 
list_cities = cities_list.to_list()

# Randomized list of 500 cities. This is probably better for plotting and to avoid hitting API limit
random_cities = random.sample(list_cities,500) # change to 500 or any number you want

# Shortened list of cities. Use this for testing code to avoid hitting API limit
short_cities = cities_list.head(3).tolist()


In [9]:
random_cities

['Merrick',
 'Veracruz',
 'Nesconset',
 'Goldsboro',
 'Teoloyucan',
 'Lenoir',
 'Bacalar',
 'Callaway',
 'Powder Springs',
 'Ewing',
 'Natick',
 'Zapotiltic',
 'South Bradenton',
 'Kearny',
 'Revere',
 'Steubenville',
 'Redwood City',
 'Roy',
 'West Lafayette',
 'Mastic',
 'Moses Lake',
 'Ashland',
 'South Saint Paul',
 'Justice',
 'Ottawa',
 'Pottsville',
 'Shrewsbury',
 'Havre De Grace',
 'Langley Park',
 'Eufaula',
 'West Springfield',
 'Alhambra',
 'Yeadon',
 'Red Bluff',
 'Coffeyville',
 'El Mirage',
 'La Palma',
 'West Melbourne',
 'East Palo Alto',
 'Calvillo',
 'Rancho Santa Margarita',
 'Coconut Creek',
 'Kingsville',
 'Ometepec',
 'Williams Lake',
 'Forrest City',
 'Ogden',
 'Jeannette',
 'Hialeah Gardens',
 'Sachse',
 'University City',
 'Encinitas',
 'Ionia',
 'Moorpark',
 'Tepatepec',
 'Cabo Rojo',
 'Hopkinton',
 'Missoula',
 'Cueramaro',
 'New Rochelle',
 'Inver Grove Heights',
 'Glenvar Heights',
 'West Bend',
 'Prospect Heights',
 'Vermilion',
 'Burnsville',
 'Burlingam

In [10]:
# Acessing the open weather API to get lattitude and longitude values

url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

# Build partial query URL
query_url = f"{url}appid={api_key}&units={units}&q="

# Making API calls
lat = []
lon = []

# loop through list of cities
for city in random_cities:
    #print (city)
    
    response = requests.get(query_url + city).json()
    try:
        #print (response)
        lat.append(response['coord']['lat'])
        lon.append(response['coord']['lon'])
    except Exception as e:
        print(f'city not found or {e}')
        pass

city not found or 'coord'
city not found or 'coord'
city not found or 'coord'
city not found or 'coord'
city not found or 'coord'
city not found or 'coord'
city not found or 'coord'
city not found or 'coord'
city not found or 'coord'
city not found or 'coord'
city not found or 'coord'
city not found or 'coord'


In [11]:
# Accessing the open weather API to get air pollution values

air_url = 'http://api.openweathermap.org/data/2.5/air_pollution?' # current air pollution values

coord = [] # latitude and longitude coordinates
aqi = [] # air quality index
co = [] # Concentration of CO (Carbon monoxide), μg/m3
nh3 = [] # Concentration of NO (Nitrogen monoxide), μg/m3
no = [] # Сoncentration of NO2 (Nitrogen dioxide), μg/m3
no2 = [] # Сoncentration of O3 (Ozone), μg/m3
o3 = [] # Сoncentration of SO2 (Sulphur dioxide), μg/m3
pm10 = [] # Сoncentration of PM2.5 (Fine particles matter),
pm2_5 = [] # Сoncentration of PM10 (Coarse particulate matter), μg/m3
so2 = [] # Сoncentration of NH3 (Ammonia), μg/m3


for lt,ln in zip(lat,lon):
    response = requests.get(f'{air_url}lat={lt}&lon={ln}&appid={api_key}').json()
    #pprint (response)
    try:
        coord.append(response['coord'])
        aqi.append(response['list'][0]['main']['aqi'])
        co.append(response['list'][0]['components']['co'])
        nh3.append(response['list'][0]['components']['nh3'])
        no.append(response['list'][0]['components']['no'])
        no2.append(response['list'][0]['components']['no2'])
        o3.append(response['list'][0]['components']['o3'])
        pm10.append(response['list'][0]['components']['pm10'])
        pm2_5.append(response['list'][0]['components']['pm2_5'])
        so2.append(response['list'][0]['components']['so2'])
    except Exception as e:
        print (f'failed to get values for lat:{lt} and lon:{ln}. Error is: {e}')

In [12]:
response

{'coord': {'lon': -96.7836, 'lat': 32.7668},
 'list': [{'main': {'aqi': 4},
   'components': {'co': 280.38,
    'no': 0.86,
    'no2': 8.4,
    'o3': 143.05,
    'so2': 2.95,
    'pm2_5': 18.61,
    'pm10': 22.13,
    'nh3': 1.76},
   'dt': 1729026424}]}

In [147]:
# Create a DataFrame
map_data = pd.DataFrame({
    'Lat': lat,
    'Lon': lon,
    'AQI': aqi,
    'CO': co,
    'NH3': nh3,
    'NO2': no2,
    'O3': o3,
    'PM10': pm10,
    'PM2.5': pm2_5,
    'SO2': so2})



map_data.head()


,Lat,Lon,AQI,CO,NH3,NO2,O3,PM10,PM2.5,SO2
0,40.6629,-73.5515,1,263.69,0.95,17.48,50.07,2.96,1.04,4.17
1,19.3333,-96.6667,2,213.62,0.00,0.86,85.12,13.78,11.95,2.74
2,40.8520,-73.1540,2,216.96,0.13,1.99,68.67,0.74,0.50,0.42
3,35.3849,-77.9928,2,233.65,0.75,1.29,92.98,4.54,3.94,1.59
4,19.7456,-99.1833,4,243.66,0.00,5.36,147.34,23.58,22.11,75.34


In [171]:
map_data_copy = map_data.copy()
map_data_copy["Lat"] = map_data_copy["Lat"].astype(str)
map_data_copy["Lon"] = map_data_copy["Lon"].astype(str)

In [172]:
map_data_copy.dtypes

Lat       object
Lon       object
AQI        int64
CO       float64
NH3      float64
NO2      float64
O3       float64
PM10     float64
PM2.5    float64
SO2      float64
dtype: object

In [207]:
app = Dash(__name__)

In [208]:
# App layout
app.layout = html.Div([

    html.H1("Map of North America Showing Air Quality Data", style={'text-align': 'center'}),

    dcc.Dropdown(id="air_quality_selector",
                 options=map_data_copy.columns.values[2:10],
                 multi=True,
                 value="AQI",
                 style={'width': "40%"},
                 ),

    html.Div(id='output_container', children=[]),
    html.Br(),

    dcc.Graph(id='aq_map', figure={})

])

In [209]:
# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='aq_map', component_property='figure')],
    [Input(component_id='air_quality_selector', component_property='value')]
)
def update_graph(chosen_airquality):

    container = "Displaying Air Quality Data for: {}".format(chosen_airquality)



    # Plotly Express
    fig = px.choropleth(
        data_frame=map_data_copy,
        locationmode='USA-states',
        # lat=map_data_copy['Lat'],
        # lon=map_data_copy['Lon'],
        color= chosen_airquality,
        hover_name= chosen_airquality,
        color_continuous_scale=px.colors.sequential.YlOrRd,
        labels={chosen_airquality: 'Scale'},
        scope= 'north america'
    )

    return container, fig


In [210]:
if __name__ == '__main__':
    app.run_server(debug=True)